## Import Libraries

In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
from scipy.sparse import hstack

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Praneet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Praneet\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 1. Data Collection and Preprocessing

In [2]:
#from google.colab import files
data = pd.read_csv("data job posts.csv")
data

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18996,Technolinguistics NGO\r\n\r\n\r\nTITLE: Senio...,"Dec 28, 2015",Senior Creative UX/ UI Designer,Technolinguistics NGO,NaN,Full-time,NaN,NaN,NaN,Long-term,...,Competitive,"To apply for this position, please send your\r...",29 December 2015,28 January 2016,NaN,As a company Technolinguistics has a mandate t...,NaN,2015,12,False
18997,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Dec 30, 2015",Category Development Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,Full-time,All interested professionals.,NaN,ASAP,Long-term with a probation period of 3 months.,...,NaN,All interested candidates are kindly requested...,30 December 2015,20 January 2016,NaN,NaN,NaN,2015,12,False
18998,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Dec 30, 2015",Operational Marketing Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,Full-time,All interested professionals.,NaN,ASAP,Long-term with a probation period of 3 months.,...,NaN,All interested candidates are kindly requested...,30 December 2015,20 January 2016,NaN,NaN,NaN,2015,12,False
18999,San Lazzaro LLC\r\n\r\n\r\nTITLE: Head of O...,"Dec 30, 2015",Head of Online Sales Department,San Lazzaro LLC,NaN,NaN,NaN,NaN,NaN,Long-term,...,Highly competitive,Interested candidates can send their CVs to:\r...,30 December 2015,29 January 2016,NaN,San Lazzaro LLC works with several internation...,NaN,2015,12,False


In [3]:
data.columns

Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')

In [4]:
data.JobDescription.isna().sum()

3892

In [5]:
data.JobRequirment.isna().sum()

2522

In [6]:
# Clean Company column
data['Company'] = data['Company'].str.strip()

# Clean RequiredQual column
data['RequiredQual'] = data['RequiredQual'].str.strip()

# Clean Eligibility column
data['Eligibility'] = data['Eligibility'].fillna('All interested candidates.')
data['Eligibility'] = data['Eligibility'].str.strip()

# Clean Title column
data['Title'] = data['Title'].str.strip()

# Clean JobDescription column
data['JobDescription'] = data['JobDescription'].str.strip()

# Clean JobRequirment column
data['JobRequirment'] = data['JobRequirment'].str.strip()

# Fill NaN with placeholder
data['JobDescription'].fillna('missing', inplace=True)
data['JobRequirment'].fillna('missing', inplace=True)

data

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,All interested candidates.,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,All interested candidates.,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,All interested candidates.,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,All interested candidates.,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,All interested candidates.,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18996,Technolinguistics NGO\r\n\r\n\r\nTITLE: Senio...,"Dec 28, 2015",Senior Creative UX/ UI Designer,Technolinguistics NGO,NaN,Full-time,All interested candidates.,NaN,NaN,Long-term,...,Competitive,"To apply for this position, please send your\r...",29 December 2015,28 January 2016,NaN,As a company Technolinguistics has a mandate t...,NaN,2015,12,False
18997,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Dec 30, 2015",Category Development Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,Full-time,All interested professionals.,NaN,ASAP,Long-term with a probation period of 3 months.,...,NaN,All interested candidates are kindly requested...,30 December 2015,20 January 2016,NaN,NaN,NaN,2015,12,False
18998,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Dec 30, 2015",Operational Marketing Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,Full-time,All interested professionals.,NaN,ASAP,Long-term with a probation period of 3 months.,...,NaN,All interested candidates are kindly requested...,30 December 2015,20 January 2016,NaN,NaN,NaN,2015,12,False
18999,San Lazzaro LLC\r\n\r\n\r\nTITLE: Head of O...,"Dec 30, 2015",Head of Online Sales Department,San Lazzaro LLC,NaN,NaN,All interested candidates.,NaN,NaN,Long-term,...,Highly competitive,Interested candidates can send their CVs to:\r...,30 December 2015,29 January 2016,NaN,San Lazzaro LLC works with several internation...,NaN,2015,12,False


## 2. Feature Engineering

In [7]:
# selecting only IT Jobs
df = data[data['IT']]
# selecting
cols = ["Company", 'RequiredQual', 'Eligibility', 'Title', 'JobDescription', 'JobRequirment']
df=df[cols]
df.head(5)

,Company,RequiredQual,Eligibility,Title,JobDescription,JobRequirment
4,Yerevan Brandy Company,- University degree; economical background is ...,All interested candidates.,Software Developer,missing,- Rendering technical assistance to Database M...
15,NetCall Communications,"- Excellent knowledge of Windows 2000 Server, ...",All interested candidates.,Network Administrator,missing,- Network monitoring and administration;\r\n- ...
19,Xalt LLC,"As a GD you are creative, innovative and have\...",All interested candidates.,Graphic Designer,The position of Graphic Designer (GD) demands ...,Graphic Designer will be responsible for every...
34,Caucasus Research Resource Center - Armenia,Participants should be mid-level professionals...,All interested candidates.,Demographic Analysis Workshop,missing,missing
35,CIT Ltd.,- Work experience of at least two years; \r\n-...,All interested candidates.,Programmer,missing,missing


In [8]:
df.columns

Index(['Company', 'RequiredQual', 'Eligibility', 'Title', 'JobDescription',
       'JobRequirment'],
      dtype='object')

In [9]:
df.Company.value_counts()

Mentor Graphics Development Services CJSC    151
Energize Global Services CJSC                 93
Lycos Europe                                  64
CQGI MA                                       61
Boomerang Software LLC                        60
                                            ... 
''Be Interactive'' LLC                         1
Corporate Image Center                         1
ArWest Communications                          1
Elanjh LLC                                     1
Yerevan Brandy Company                         1
Name: Company, Length: 996, dtype: int64

In [10]:
df.Title.value_counts()

Software Developer                      134
Web Developer                           101
Java Developer                           88
Graphic Designer                         75
Software Engineer                        69
                                       ... 
Computer Programmer                       1
Unix/ Oracle Administrator Assistant      1
Junior .Net Developer                     1
Food Security Specialist                  1
JavaScript Professional                   1
Name: Title, Length: 1272, dtype: int64

In [11]:
df.RequiredQual.unique

<bound method Series.unique of 4        - University degree; economical background is ...
15       - Excellent knowledge of Windows 2000 Server, ...
19       As a GD you are creative, innovative and have\...
34       Participants should be mid-level professionals...
35       - Work experience of at least two years; \r\n-...
                               ...                        
18987    - Experience in shipping beautifully designed ...
18988    - Expert knowledge of JavaScript, especially B...
18990    - Master's degree in IT, Engineering or a rela...
18991    - Master's degree in IT, Engineering or a rela...
18992    - 2 - 5 years of experience in software develo...
Name: RequiredQual, Length: 3759, dtype: object>

In [12]:
# Modifying job Titles (Selecting only top 21 job titles to manage class imbalance)
classes = df['Title'].value_counts()[:21]
keys = classes.keys().to_list()

df = df[df['Title'].isin(keys)]
df['Title'].value_counts()

Software Developer           134
Web Developer                101
Java Developer                88
Graphic Designer              75
Software Engineer             69
Senior Java Developer         69
PHP Developer                 65
Senior Software Engineer      63
Programmer                    56
IT Specialist                 55
Senior QA Engineer            43
Senior Software Developer     41
Android Developer             37
.NET Developer                36
Senior .NET Developer         34
Senior PHP Developer          34
iOS Developer                 31
Software QA Engineer          29
Senior Web Developer          29
Database Developer            29
Database Administrator        28
Name: Title, dtype: int64

In [13]:
# Change job titles to base title. For exmaple, changing Senior Java Developer to Java Developer.

def change_titles(x):
    x = x.strip()
    if x == 'Senior Java Developer':
        return 'Java Developer'
    elif x == 'Senior Software Engineer':
        return 'Software Engineer'
    elif x == 'Senior QA Engineer':
        return 'Software QA Engineer'
    elif x == 'Senior Software Developer':
        return 'Senior Web Developer'
    elif x =='Senior PHP Developer':
        return 'PHP Developer'
    elif x == 'Senior .NET Developer':
        return '.NET Developer'
    elif x == 'Senior Web Developer':
        return 'Web Developer'
    elif x == 'Database Administrator':
        return 'Database Admin/Dev'
    elif x == 'Database Developer':
        return 'Database Admin/Dev'
    else:
        return x


df['Title'] = df['Title'].apply(change_titles)
df['Title'].value_counts()

Java Developer          157
Software Developer      134
Software Engineer       132
Web Developer           130
PHP Developer            99
Graphic Designer         75
Software QA Engineer     72
.NET Developer           70
Database Admin/Dev       57
Programmer               56
IT Specialist            55
Senior Web Developer     41
Android Developer        37
iOS Developer            31
Name: Title, dtype: int64

In [14]:
df.head()

,Company,RequiredQual,Eligibility,Title,JobDescription,JobRequirment
4,Yerevan Brandy Company,- University degree; economical background is ...,All interested candidates.,Software Developer,missing,- Rendering technical assistance to Database M...
19,Xalt LLC,"As a GD you are creative, innovative and have\...",All interested candidates.,Graphic Designer,The position of Graphic Designer (GD) demands ...,Graphic Designer will be responsible for every...
35,CIT Ltd.,- Work experience of at least two years; \r\n-...,All interested candidates.,Programmer,missing,missing
76,Ameria CJSC,- Professional background in graphic design;\r...,All interested candidates.,Graphic Designer,Ameria CJSC is seeking qualified candidates fo...,- Design of company production packaging;\r\n-...
92,"Synergy International Systems, Inc./Armenia","- Degree in Computer Science, Information Tech...",All interested candidates.,Software Developer,"Synergy International Systems, Inc./Armenia se...",Specific tasks and key responsibilities includ...


In [15]:
df.columns

Index(['Company', 'RequiredQual', 'Eligibility', 'Title', 'JobDescription',
       'JobRequirment'],
      dtype='object')

In [16]:
# One-hot encode on Company and Title columns
categorical_cols = ['Company', 'Title']
onehot = OneHotEncoder()
df_onehot = onehot.fit_transform(df[categorical_cols])

# TF-IDF on text columns
tfidf = TfidfVectorizer()
df_desc_tfidf = tfidf.fit_transform(df['JobDescription'])
df_req_tfidf = tfidf.fit_transform(df['JobRequirment'])

X = hstack([df_onehot, df_desc_tfidf, df_req_tfidf])
y = df['Title']

In [17]:
# Checking if One hot encode and TF-IDF are executed properly
print(df['Company'].dtype)
print(df['Title'].dtype)
print(df_onehot.shape)

object
object
(1146, 469)


## 3. Model Selection and Training

In [18]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Collaborative Filtering with KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Content-based Filtering
content_model = cosine_similarity(X_test, X_train)

# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Logistic Regression
logistic_model = LogisticRegression(max_iter=15, verbose=1, C=0.75)
logistic_model.fit(X_train, y_train)

# Neural Network
mlp_model = MLPClassifier(hidden_layer_sizes=(128,), max_iter=5, random_state=42)
mlp_model.fit(X_train, y_train)

# Model evaluation
knn_pred = knn.predict(X_test)
print("KNN F1 Score:", f1_score(y_test, knn_pred, average='weighted'))

content_pred = np.argmax(content_model, axis=1)
print("Content-Based Accuracy Score:", accuracy_score(y_test, content_pred))

nb_pred = nb_model.predict(X_test)
print("Naive Bayes F1 Score:", f1_score(y_test, nb_pred, average='weighted'))

logistic_pred = logistic_model.predict(X_test)
print("Logistic Regression F1 Score:", f1_score(y_test, logistic_pred, average='weighted'))

mlp_pred = mlp_model.predict(X_test)
print("Neural Network F1 Score:", f1_score(y_test, mlp_pred, average='weighted'))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Praneet\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s finished
C:\Users\Praneet\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


KNN F1 Score: 0.9054175706582613
Content-Based Accuracy Score: 0.0
Naive Bayes F1 Score: 0.8978978230897458
Logistic Regression F1 Score: 1.0
Neural Network F1 Score: 0.726380780515267


KNN F1 Score: 0.91: The model achieved a high F1 score, indicating good performance in terms of precision and recall.

Content-Based Accuracy Score: 0.0: The accuracy suggests that there might be an issue with the implementation or the nature of the content-based recommendations.

Naive Bayes F1 Score: 0.90: The model has a high F1 score, indicating good performance.

Logistic Regression F1 Score: 1.0: The model achieved a perfect F1 score of 1.0 on the test set. While this might seem promising, it could also be an indication of overfitting.

Neural Network F1 Score: 0.73: The model achieved a relatively lower F1 score compared to other models. This might suggest that the neural network needs further tuning, such as increasing the number of epochs or changing the architecture.

## 4. Recommendation Engine

In [19]:
# Sample user profile data
user_profiles = [{
  'experience': '5 years',
  'skills': ['Python', 'SQL', 'AWS'],
  'education': 'BTech CS'
},
{
  'experience': '10 years',
  'skills': ['Java', 'springboot', 'microservices'],
  'education': 'MTech CSE'
}]

In [20]:
# Recommendation function
def get_recommendations(user_profile, job_descriptions):
    # Extract keywords from user profile
    keywords = extract_keywords(user_profile)

    # Get most similar jobs based on keywords
    similar_jobs = find_similar_jobs(keywords, job_descriptions)

    # Rank jobs based on similarity score
    ranked_jobs = rank_jobs(similar_jobs)

    return ranked_jobs[:5]  # Return top 5

# Helper functions
def extract_keywords(user_profile):
    keywords = []
    keywords.extend(user_profile['skills'])
    keywords.append(user_profile['education'])
    keywords.append(user_profile['experience'])
    return ' '.join(keywords)  # Combine keywords into a single string

def find_similar_jobs(keywords, job_descriptions):
    # Use TF-IDF to vectorize job descriptions and user keywords
    vectorizer = TfidfVectorizer()
    job_matrix = vectorizer.fit_transform(job_descriptions)
    user_vector = vectorizer.transform([keywords])

    # Compute cosine similarity
    similarities = cosine_similarity(user_vector, job_matrix).flatten()

    # Combine job indices with similarity scores
    similar_jobs = list(enumerate(similarities))

    return similar_jobs

def rank_jobs(similar_jobs):
    return sorted(similar_jobs, key=lambda x: x[1], reverse=True)

In [21]:
# User Profile 1
user_profile = user_profiles[0]
job_descriptions = df['JobDescription']

recommendations = get_recommendations(user_profile, job_descriptions)

# Print job titles and similarity scores with rankings
for rank, (idx, score) in enumerate(recommendations[:5], start=1):
    job_title = df.loc[df.index[idx], 'Title']
    print(f"Rank: {rank}, Job Title: {job_title}, Similarity Score: {score}")

Rank: 1, Job Title: iOS Developer, Similarity Score: 0.24881449987410953
Rank: 2, Job Title: Software Engineer, Similarity Score: 0.1464317942139053
Rank: 3, Job Title: Senior Web Developer, Similarity Score: 0.144294954079096
Rank: 4, Job Title: Senior Web Developer, Similarity Score: 0.14027241062781237
Rank: 5, Job Title: Graphic Designer, Similarity Score: 0.1357330187626437


In [22]:
# User Profile 2
user_profile = user_profiles[1]
job_descriptions = df['JobDescription']
recommendations = get_recommendations(user_profile, job_descriptions)

# Print job titles and similarity scores with rankings
for rank, (idx, score) in enumerate(recommendations[:5], start=1):
    job_title = df.loc[df.index[idx], 'Title']
    print(f"Rank: {rank}, Job Title: {job_title}, Similarity Score: {score}")


Rank: 1, Job Title: iOS Developer, Similarity Score: 0.2800158126091371
Rank: 2, Job Title: Java Developer, Similarity Score: 0.1905458940252372
Rank: 3, Job Title: Java Developer, Similarity Score: 0.18943460247902108
Rank: 4, Job Title: Java Developer, Similarity Score: 0.18943460247902108
Rank: 5, Job Title: Java Developer, Similarity Score: 0.18943460247902108


## 5. Evaluation

In [23]:
# Evaluate performance on the testing dataset
knn_pred = knn.predict(X_test)
content_pred = np.argmax(content_model, axis=1)
nb_pred = nb_model.predict(X_test)
logistic_pred = logistic_model.predict(X_test)
mlp_pred = mlp_model.predict(X_test)

# LabelEncoder for converting string labels to numerical labels
label_encoder = LabelEncoder()
y_test_numeric = label_encoder.fit_transform(y_test)

# Precision, Recall, and F1-score for KNN
precision_knn = precision_score(y_test, knn_pred, average='weighted')
recall_knn = recall_score(y_test, knn_pred, average='weighted')
f1_knn = f1_score(y_test, knn_pred, average='weighted')

print("KNN Precision:", precision_knn)
print("KNN Recall:", recall_knn)
print("KNN F1 Score:", f1_knn)

# Precision, Recall, and F1-score for Content-Based Filtering
precision_content = precision_score(y_test_numeric, content_pred, average='weighted')
recall_content = recall_score(y_test_numeric, content_pred, average='weighted')
f1_content = f1_score(y_test_numeric, content_pred, average='weighted')

print("Content-Based Precision:", precision_content)
print("Content-Based Recall:", recall_content)
print("Content-Based F1 Score:", f1_content)

# Precision, Recall, and F1-score for Naive Bayes
precision_nb = precision_score(y_test, nb_pred, average='weighted')
recall_nb = recall_score(y_test, nb_pred, average='weighted')
f1_nb = f1_score(y_test, nb_pred, average='weighted')

print("Naive Bayes Precision:", precision_nb)
print("Naive Bayes Recall:", recall_nb)
print("Naive Bayes F1 Score:", f1_nb)

# Precision, Recall, and F1-score for Logistic Regression
precision_logistic = precision_score(y_test, logistic_pred, average='weighted')
recall_logistic = recall_score(y_test, logistic_pred, average='weighted')
f1_logistic = f1_score(y_test, logistic_pred, average='weighted')

print("Logistic Regression Precision:", precision_logistic)
print("Logistic Regression Recall:", recall_logistic)
print("Logistic Regression F1 Score:", f1_logistic)

# Precision, Recall, and F1-score for Neural Network
precision_mlp = precision_score(y_test, mlp_pred, average='weighted')
recall_mlp = recall_score(y_test, mlp_pred, average='weighted')
f1_mlp = f1_score(y_test, mlp_pred, average='weighted')

print("Neural Network Precision:", precision_mlp)
print("Neural Network Recall:", recall_mlp)
print("Neural Network F1 Score:", f1_mlp)


KNN Precision: 0.913195860435936
KNN Recall: 0.9043478260869565
KNN F1 Score: 0.9054175706582613
Content-Based Precision: 0.06521739130434782
Content-Based Recall: 0.004347826086956522
Content-Based F1 Score: 0.008152173913043478
Naive Bayes Precision: 0.9248444540271672
Naive Bayes Recall: 0.908695652173913
Naive Bayes F1 Score: 0.8978978230897458
Logistic Regression Precision: 1.0
Logistic Regression Recall: 1.0
Logistic Regression F1 Score: 1.0
Neural Network Precision: 0.8068630586337459
Neural Network Recall: 0.7521739130434782
Neural Network F1 Score: 0.726380780515267


C:\Users\Praneet\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Praneet\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Praneet\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
